**Recommendation system based on Matrix Factorization** 
* Dataset link : https://www.kaggle.com/rounakbanik/the-movies-dataset

# Libraries and data

In [1]:
import numpy as np
import sklearn
import pandas as pd
from sklearn import preprocessing 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import progressbar as pb
import random
import numpy
from matplotlib import pyplot as plt

In [3]:
## mini imdb dataset (from kaggle)
file_path = '/content/drive/My Drive/Colab Notebooks/Recommendation systems/Data/movies dataset (huge)/ratings_small.csv'
#file_path = '/Data/ratings.csv'
df = pd.read_csv(file_path)  #, error_bad_lines=False ,warn_bad_lines=False)
print("Rows and columns : " , df.shape)

Rows and columns :  (100004, 4)


In [4]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


# Data preprocessing

In [6]:
data = df[['userId' , 'movieId','rating']]
#print(data)
X = df[['userId' , 'movieId']]
y = df['rating']

In [8]:
n = 1000
X1 = df[['userId']]  [0:n]
X2 = df[['movieId']] [0:n]
y1 = df[['rating']] [0:n]
#print(X2['movieId'])
X1.head()

,userId
0,1
1,1
2,1
3,1
4,1


In [9]:
c_data = pd.DataFrame()
movies = []
c_data['id'] = 0
for movie in pb.progressbar(X2['movieId']):
    if movie not in movies:
        movies.append(movie)
        c_data[movie] = 0
users = []
for user in pb.progressbar(X1['userId']):
    if user not in users:
        users.append(user)
        append_dic = {'id':user} 
        for column in c_data.columns:
            if column != 'id':
                append_dic[column] = 0
        c_data = c_data.append(pd.DataFrame([append_dic]))
c_data = c_data.set_index('id')

100% (1000 of 1000) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1000 of 1000) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [10]:
for index in pb.progressbar(range(len(X1)-1)):
    c_data.loc[X1['userId'][index],X2['movieId'][index]] = y1['rating'][index]

100% (999 of 999) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [11]:
c_data

,31,1029,1061,1129,1172,1263,1287,1293,1339,1343,1371,1405,1953,2105,2150,2193,2294,2455,2968,3671,10,17,39,47,50,52,62,110,144,150,153,161,165,168,185,186,208,222,223,225,...,4321,4878,7502,54286,64614,69757,78499,81834,88125,93363,2038,2394,2720,2724,2861,3157,3175,3354,3623,3986,3988,2,5,6,11,14,16,19,22,25,44,82,94,95,101,107,123,125,145,149
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2.5,3.0,3.0,2.0,4.0,2.0,2.0,2.0,3.5,2.0,2.5,1.0,4.0,4.0,3.0,2.0,2.0,2.5,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,4.0,4.0,3.0,3.0,4.0,3.0,5.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
c_data.to_csv ('final_data.csv', index = None, header=True)

# Matrix Factorization

In [13]:
#final_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Recommendation systems/Data/MF-Data/final_data(movies dataset).csv')
final_data = c_data
print(final_data.shape)
final_data.head()

(15, 669)


,31,1029,1061,1129,1172,1263,1287,1293,1339,1343,1371,1405,1953,2105,2150,2193,2294,2455,2968,3671,10,17,39,47,50,52,62,110,144,150,153,161,165,168,185,186,208,222,223,225,...,4321,4878,7502,54286,64614,69757,78499,81834,88125,93363,2038,2394,2720,2724,2861,3157,3175,3354,3623,3986,3988,2,5,6,11,14,16,19,22,25,44,82,94,95,101,107,123,125,145,149
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2.5,3.0,3.0,2.0,4.0,2.0,2.0,2.0,3.5,2.0,2.5,1.0,4.0,4.0,3.0,2.0,2.0,2.5,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,4.0,4.0,3.0,3.0,4.0,3.0,5.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,4.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Matrix Factorization 

In [14]:
def MF(R, P, Q, K, steps=1000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in pb.progressbar(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
    return P, Q.T


In [15]:
dim = final_data.shape
R = np.array(final_data) #final_data.to_numpy()
N = len(R)
M = len(R[0])
K = 2 
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

print(P.shape)
print(Q.shape)
print(R.shape)

(15, 2)
(669, 2)
(15, 669)


In [16]:
#hyperparametrs
K = 2
steps = 1000 #1000
alpha = 0.002
beta = 0.02

In [17]:
nP, nQ = MF(R, P, Q, K , steps , alpha , beta)
nR = numpy.dot(nP, nQ.T)
nR

100% (1000 of 1000) |####################| Elapsed Time: 0:00:41 Time:  0:00:41


array([[2.47544559, 2.96639655, 2.96743233, ..., 2.69727214, 2.66887655,
        3.38900285],
       [2.97717394, 2.56395329, 2.00650669, ..., 2.64720629, 2.76049429,
        5.63243755],
       [3.26793719, 3.45979904, 3.20719025, ..., 3.28950066, 3.31903877,
        5.18155184],
       ...,
       [3.05129708, 2.80402   , 2.33079293, ..., 2.81789571, 2.90811611,
        5.49936223],
       [2.99080703, 3.74840636, 3.84119109, ..., 3.35658473, 3.29812185,
        3.8395408 ],
       [3.33802595, 3.74068445, 3.59770612, ..., 3.48293809, 3.48274849,
        4.97215245]])

In [18]:
## rmse (A : Orginal , B : estimated)
def rmse( A , B ):
  count = 0
  E = 0
  sh = A.shape
  for i in range(0,sh[0]) :
    for j in range(0,sh[1]) : 
      if (A[i][j] != 0 ) :
        E +=  ( A[i][j] - B[i][j] ) ** 2
        count += 1
  
  rmse = (E/count)**0.5
  print("RMSE value : " , rmse)

rmse(R,nR)

RMSE value :  0.267276082503285
